# Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.filterwarnings("ignore")

# Loading Data

In [19]:
features = ['mapped_veh_id', 'timestamps_UTC', 'lat', 'lon',
            'RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2',
            'RS_E_OilPress_PC1', 'RS_E_OilPress_PC2',
            'RS_E_RPM_PC1', 'RS_E_RPM_PC2',
            'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2',
            'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2',
            'temperature', 'humidity', 'rain', 'snow_depth', 'weather_code', 'cloud_cover', 'evapotranspiration', 'wind_speed']
merged_data = pd.read_csv('merged_data.csv', sep=';', index_col=[0], chunksize=1000000)  # len = 17679273
raw_data = pd.DataFrame(columns=features)

for data in merged_data:
    raw_data = pd.concat([raw_data, data], axis=0)

          mapped_veh_id       timestamps_UTC        lat       lon  \
0                 180.0  2022-08-22 14:31:20  51.033240  3.687010   
1                 180.0  2022-08-22 14:32:23  51.018234  3.676889   
2                 180.0  2022-08-22 14:33:24  51.004336  3.665953   
3                 180.0  2022-08-22 14:34:24  50.997092  3.651049   
4                 180.0  2022-08-22 14:35:24  50.996757  3.650107   
...                 ...                  ...        ...       ...   
17679268          115.0  2023-09-13 21:51:55  50.144768  4.500417   
17679269          191.0  2023-09-13 21:51:57  51.150134  4.614944   
17679270          120.0  2023-09-13 21:52:16  50.145201  4.500263   
17679271          196.0  2023-09-13 21:52:22  50.419692  4.535372   
17679272          191.0  2023-09-13 21:52:58  51.151427  4.620015   

          RS_E_InAirTemp_PC1  RS_E_InAirTemp_PC2  RS_E_OilPress_PC1  \
0                       46.0                44.0              458.0   
1                       45.0 

# Preprocessing Data

In [4]:
def train_test_split(data):
    data_len = len(data)
    train_data, valid_data, test_data = data.iloc[:int(0.64*data_len)], data.iloc[int(0.64*data_len): int(0.8*data_len)], data.iloc[int(0.8*data_len):]
    return train_data, valid_data, test_data


def generate_label(raw_data):
    label_dict = {}
    keys = ['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1', 'RS_E_OilPress_PC2', 'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2']

    for var in keys:
        raw_feature = raw_data[var]
        if 'AirTemp' in var:
            label_dict[var] = np.where((raw_feature > 65) | (raw_feature == 0), 1, 0)
        elif 'WatTemp' in var:
            label_dict[var] = np.where((raw_feature > 100) | (raw_feature == 0), 1, 0)
        elif 'OilTemp' in var:
            label_dict[var] = np.where((raw_feature > 115) | (raw_feature == 0), 1, 0)
        else:
            stats_dict = compute_stats(raw_feature)
            mean, std = stats_dict['mean'], stats_dict['std']
            label_dict[var] = np.where((raw_feature > mean+3*std) | (raw_feature == 0), 1, 0)

    return label_dict


def compute_stats(feature):
    length = len(feature)
    mean = np.mean(feature)
    std = np.std(feature)
    min = np.min(feature)
    max = np.max(feature)

    stats_dict = {'length': length, 'mean': mean,
                  'std': std, 'min': min, 'max': max}

    return stats_dict


def preprocess(feature):
    stats_dict = compute_stats(feature)
    mean, std = stats_dict['mean'], stats_dict['std']
    new_feature = feature.copy()
    new_feature.dropna()
    new_feature.drop(np.where((feature > mean + 3*std) |
                     (feature < mean - 3*std) | (feature == 0))[0], inplace=True)

    return new_feature


def plot_stats(feature, key):
    stats_dict = compute_stats(feature)
    mean, std = stats_dict['mean'], stats_dict['std']
    plt.hist(feature, bins=200)
    plt.xlim(0, mean+4*std)
    plt.title(key)


def preprocess_feature(data):
    keys = ['RS_E_RPM_PC1', 'RS_E_RPM_PC2', 'RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1',
            'RS_E_OilPress_PC2', 'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2']
    # new_data = data.dropna()
    # new_data.reset_index(drop=True, inplace=True)
    # new_data = data.dropna()
    drop_index = []
    rmp_feature_pc1 = data[keys[0]]
    rmp_feature_pc2 = data[keys[1]]
    for var in keys[2:]:
        feature = data[var]
        stats_dict = compute_stats(feature)
        mean, std = stats_dict['mean'], stats_dict['std']
        drop_index.extend(list(np.where((feature > mean + 6*std) | (feature < mean - 6*std))[0]))

    data.drop(np.unique(drop_index), inplace=True)


def print_stats(data):
    keys = data.keys()[4:14]
    for var in keys:
        feature = data[var]
        stats_dict = compute_stats(feature)
        print(f'{var}: {stats_dict}')


def get_outlier_frac(train_data):
    label_df = generate_label(train_data)
    outlier_fracs = {}
    for feature in label_df.keys():
        df_feature = label_df[feature]
        outlier_fracs[feature] = np.sum(df_feature)/len(df_feature)    
    
    return outlier_fracs


def compute_metrics(labels, preds):
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)

    return acc, f1

In [20]:
raw_data.dropna(inplace=True)
raw_data.reset_index(drop=True, inplace=True)
label_dict = generate_label(raw_data)
label_df = pd.DataFrame(label_dict)
label_df.to_csv('labels.csv', sep=';')
train_data, valid_data, test_data = train_test_split(raw_data)
# train_data.to_csv('data/train_data.csv', sep=';')
# valid_data.to_csv('data/valid_data.csv', sep=';')
# test_data.to_csv('data/test_data.csv', sep=';')
train_label, valid_label, test_label = train_test_split(label_df)
# train_label.to_csv('data/train_label.csv', sep=';')
# valid_label.to_csv('data/valid_label.csv', sep=';')
# test_label.to_csv('data/test_label.csv', sep=';')

# sample_data = test_data.iloc[-1000000:]
# sample_data.reset_index(drop=True, inplace=True)
# sample_data.to_csv('data/sample_data.csv', sep=';')

print('Statistics of raw data:')
print_stats(train_data)
print()
preprocess_feature(train_data)
print('Statistics of processed data:')
print_stats(train_data)

Statistics of raw data:
RS_E_InAirTemp_PC1: {'length': 11306586, 'mean': 29.871796692101263, 'std': 408.3473286445919, 'min': 0.0, 'max': 65535.0}
RS_E_InAirTemp_PC2: {'length': 11306586, 'mean': 30.514459313176776, 'std': 433.0321859409222, 'min': 0.0, 'max': 65535.0}
RS_E_OilPress_PC1: {'length': 11306586, 'mean': 266.1173010328297, 'std': 115.3585440657922, 'min': 0.0, 'max': 690.0}
RS_E_OilPress_PC2: {'length': 11306586, 'mean': 277.09972469367654, 'std': 119.64415304368967, 'min': 0.0, 'max': 690.0}
RS_E_RPM_PC1: {'length': 11306586, 'mean': 907.4392847431227, 'std': 384.0923139041148, 'min': 0.0, 'max': 2309.0}
RS_E_RPM_PC2: {'length': 11306586, 'mean': 904.4026347020529, 'std': 388.72405118029866, 'min': 0.0, 'max': 6790.5}
RS_E_WatTemp_PC1: {'length': 11306586, 'mean': 75.72996871096552, 'std': 14.248612883416609, 'min': 0.0, 'max': 108.0}
RS_E_WatTemp_PC2: {'length': 11306586, 'mean': 75.06893940824337, 'std': 14.966431641706677, 'min': -17.0, 'max': 107.0}
RS_T_OilTemp_PC1: {

# Training Models

In [11]:
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans
import joblib


def create_models():
    outlier_fracs = get_outlier_frac(train_data)
    keys = ['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1', 'RS_E_OilPress_PC2', 'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2']
    # model_names = ['ellip_envelope', 'iso_forest', 'local_outlier_factor', 'kmeans']
    models = {}

    for var in keys:
        ellip_envelope =  EllipticEnvelope(contamination = outlier_fracs[var])
        iso_forest = IsolationForest(contamination=outlier_fracs[var])
        local_outlier_factor = LocalOutlierFactor(n_neighbors=20, contamination=outlier_fracs[var], novelty=True)
        kmeans = KMeans(n_clusters=2, n_init='auto')
        
        models[f'ellip_envelope_{var}'] = ellip_envelope
        models[f'iso_forest_{var}'] = iso_forest
        models[f'local_outlier_factor_{var}'] = local_outlier_factor
        models[f'kmeans_{var}'] = kmeans

    return models


def train_model(model, model_name, feature):
    X_train = train_data.iloc[:int(0.1*len(train_data))][feature].values.reshape(-1, 1) 
    model.fit(X_train)

    joblib.dump(model, f'models/{model_name}_{feature}.joblib')


def test_model(model_name, feature):
    model = joblib.load(f'models/{model_name}_{feature}.joblib')
    X_test = test_data[feature].values.reshape(-1, 1)
    preds = model.predict(X_test)
    preds = np.where(preds==-1, 1, 0)
    acc, f1 = compute_metrics(test_label[feature], preds)
    print(feature, acc, f1)
    np.save(f'preds/preds_{model_name}_{feature}.npy', preds)

In [12]:
models = create_models()
# print(models.keys())
model_names = ['ellip_envelope', 'iso_forest', 'local_outlier_factor', 'kmeans']
keys = ['RS_E_InAirTemp_PC1', 'RS_E_InAirTemp_PC2', 'RS_E_OilPress_PC1', 'RS_E_OilPress_PC2', 'RS_E_WatTemp_PC1', 'RS_E_WatTemp_PC2', 'RS_T_OilTemp_PC1', 'RS_T_OilTemp_PC2']

# model_name = 'ellip_envelope'
# model_name = 'iso_forest'
model_name = 'local_outlier_factor'
# model_name = 'kmeans'
# for model_name in model_names:
for var in keys:
    model = models[f'{model_name}_{var}']
    train_model(model, model_name, var)
    test_model(model_name, var)

local_outlier_factor
RS_E_InAirTemp_PC1 0.9913123930004424 0.1129349208184025
RS_E_InAirTemp_PC2 0.9902052155642204 0.32658779576587793
RS_E_OilPress_PC1 0.9561711132538931 0.0012640512843663943
RS_E_OilPress_PC2 0.9683899709875361 0.00019693850147703876
RS_E_WatTemp_PC1 0.9942368471028149 0.03652708776910339
RS_E_WatTemp_PC2 0.99168428235402 0.0031890351472384313
RS_T_OilTemp_PC1 0.9985110274816044 0.0
RS_T_OilTemp_PC2 0.9902652159774308 0.0
